# Test MiniGrid sequential tasks with PPO + BLIP_SPP

## Setup

Default Jupyter settings

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
#%load_ext autoreload
#%autoreload 2
%reload_ext autoreload
%autoreload 2

Import libraries

In [2]:
import sys, os, time
import numpy as np
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from datetime import date

import pickle
import torch
from arguments_rl import get_args

from collections import deque
from rl_module.a2c_ppo_acktr.envs import make_vec_envs
from rl_module.a2c_ppo_acktr.storage import RolloutStorage
from rl_module.train_ppo import train_ppo
from stable_baselines3.common.utils import set_random_seed
from torch_ac.utils import DictList
from torch.utils.tensorboard import SummaryWriter

# Gym MiniGrid specific
import gym
from gym import spaces
from gym.wrappers import Monitor
import gym_minigrid
from gym_minigrid.wrappers import FlatObsWrapper, ImgObsWrapper, RGBImgPartialObsWrapper, RGBImgObsWrapper
from gym.wrappers import Monitor

/Users/inigo/.local/share/virtualenvs/tfm-experiments-K5nk3NK1/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define functions and helpers

In [3]:
# define wrapper for CNN Policies
def ImgRGBImgPartialObsWrapper(env):
    return ImgObsWrapper(RGBImgPartialObsWrapper(env))

def get_tb_dir(result_dir, log_name, stage='train',task_idx=0):
    tb_dir = os.path.join(result_dir, log_name, stage, "task_"+str(task_idx))
    return tb_dir

Load arguments

In [4]:
args = {
'algo':'ppo',
'experiment':'minigrid-doorkey-wallgap-lavagap',
'approach':'blip_spp',#'fine-tuning',#'blip',
'wrapper':'flat',#'img'
'optimizer':'Adam',#'RMSProp',#'Adam',
'gail':False,
'gail_experts_dir':'./gail_experts',
'gail_batch_size':128,
'gail_epoch':5,
'lr':2.5e-4,#7e-4,#1e-4,
'eps':1e-8,#1e-5,
'gamma':0.99,
'use_gae':True,
'gae_lambda':0.95,#0.99,
'entropy_coef':0.01,
'value_loss_coef':0.5,
'max_grad_norm':0.5,
'seed':1,
'cuda_deterministic':False,
'num_processes':16,#1
'num_steps':128,#5,
'ppo_epoch':4,
'num_mini_batch':256,#8,#32,
'clip_param':0.2,#0.1,
'log_interval':10,
'save_interval':10,
'eval_interval':100,
'num_env_steps':5e5,
'env_name':'PongNoFrameskip-v4',
'log_dir':'./logs/',
'save_dir':'./trained_models/',
'no_cuda':True,
'use_proper_time_limits':False,
'recurrent_policy':False,
'use_linear_lr_decay':False,
'ewc_lambda':5000.0,
'ewc_online':True,#False,
'ewc_epochs':100,
'num_ewc_steps':20,
'save_name':None,
'date':date.today(),
'task_id':None,
'single_task':False,
'F_prior':1e-16,#5e-18,#1e-15,
'input_padding':False,
'sample':False,
'samples':1,
'num_eval_episodes':10,
'update_bits_rounding': 'floor',
'fisher_term': 'f0t',
'loss_method': 'spp',
'spp_lambda': 4,
# render arguments
'render_ckpt_path':'',
'render_task_idx':0,
'num_render_traj':1000
}

args = DictList(args)

args.cuda = not args.no_cuda and torch.cuda.is_available()

assert args.algo in ['a2c', 'ppo', 'acktr']
if args.recurrent_policy:
    assert args.algo in ['a2c', 'ppo'], \
        'Recurrent policy is not implemented for ACKTR'
conv_experiment = [
    'atari',
]

print(args)

{'algo': 'ppo', 'experiment': 'minigrid-doorkey-wallgap-lavagap', 'approach': 'blip_spp', 'wrapper': 'flat', 'optimizer': 'Adam', 'gail': False, 'gail_experts_dir': './gail_experts', 'gail_batch_size': 128, 'gail_epoch': 5, 'lr': 0.00025, 'eps': 1e-08, 'gamma': 0.99, 'use_gae': True, 'gae_lambda': 0.95, 'entropy_coef': 0.01, 'value_loss_coef': 0.5, 'max_grad_norm': 0.5, 'seed': 1, 'cuda_deterministic': False, 'num_processes': 16, 'num_steps': 128, 'ppo_epoch': 4, 'num_mini_batch': 256, 'clip_param': 0.2, 'log_interval': 10, 'save_interval': 10, 'eval_interval': 100, 'num_env_steps': 500000.0, 'env_name': 'PongNoFrameskip-v4', 'log_dir': './logs/', 'save_dir': './trained_models/', 'no_cuda': True, 'use_proper_time_limits': False, 'recurrent_policy': False, 'use_linear_lr_decay': False, 'ewc_lambda': 5000.0, 'ewc_online': True, 'ewc_epochs': 100, 'num_ewc_steps': 20, 'save_name': None, 'date': datetime.date(2023, 10, 16), 'task_id': None, 'single_task': False, 'F_prior': 1e-16, 'input_pa

In [5]:
# Split

if args.approach == 'fine-tuning' or args.approach == 'ft-fix':
    log_name = '{}_{}_{}_{}'.format(args.date, args.experiment, args.approach,args.seed)
elif args.approach == 'ewc' in args.approach:
    log_name = '{}_{}_{}_{}_lamb_{}'.format(args.date, args.experiment, args.approach, args.seed, args.ewc_lambda)
elif args.approach == 'blip':
    log_name = '{}_{}_{}_{}_F_prior_{}'.format(args.date, args.experiment, args.approach, args.seed, args.F_prior)
elif args.approach == 'blip_ewc':
    log_name = '{}_{}_{}_{}_F_prior_{}_lamb_{}'.format(args.date, args.experiment, args.approach, args.seed, args.F_prior, args.ewc_lambda)
elif args.approach == 'blip_spp':
    log_name = '{}_{}_{}_{}_{}_F_prior_{}_spp_lamb_{}'.format(args.date, args.experiment, args.approach, args.seed, args.num_env_steps, args.F_prior, args.spp_lambda)

if args.experiment in conv_experiment:
    log_name = log_name + '_conv'

# Seed
set_random_seed(args.seed)

# Inits
if args.cuda:
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

device = torch.device("cuda" if args.cuda else "cpu")

#taskcla = [(0,14), (1,18), (2,18), (3,18), (4,18), (5,6)]
#task_sequences = [(0, 'KungFuMasterNoFrameskip-v4'), (1, 'BoxingNoFrameskip-v4'), (2, 'JamesbondNoFrameskip-v4'), (3, 'KrullNoFrameskip-v4'), (4, 'RiverraidNoFrameskip-v4'), (5, 'SpaceInvadersNoFrameskip-v4')]


taskcla = [(0,7), (1,7), (2,7)]
tasks_sequence = [
    (0, 'MiniGrid-DoorKey-6x6-v0'), 
    (1, 'MiniGrid-WallGapS6-v0'), 
    (2, 'MiniGrid-LavaGapS6-v0')
    ]
# hard coded for atari environment
#obs_shape = (4,84,84)

# for FlatObsWrapper Minigrid environment
if args.wrapper == 'flat':
    wrapper_class = FlatObsWrapper
    obs_shape = (2739,)
# for ImgRGBImgPartialObsWrapper Minigrid environment
elif args.wrapper == 'img':
    wrapper_class = ImgRGBImgPartialObsWrapper
    obs_shape = (12, 56, 56)

if args.approach == 'blip':
    from rl_module.ppo_model import QPolicy
    print('using fisher prior of: ', args.F_prior)
    actor_critic = QPolicy(obs_shape,
        taskcla,
        base_kwargs={'F_prior': args.F_prior, 'recurrent': args.recurrent_policy}).to(device)
elif args.approach == 'blip_ewc':
    from rl_module.ppo_model import QPolicy
    print('using fisher prior of: ', args.F_prior)
    print('using EWC lambda of: ', args.ewc_lambda)
    actor_critic = QPolicy(obs_shape,
        taskcla,
        base_kwargs={'F_prior': args.F_prior, 'recurrent': args.recurrent_policy}).to(device)    
elif args.approach == 'blip_spp':
    from rl_module.ppo_model import QPolicy
    print('using fisher prior of: ', args.F_prior)
    print('using SPP lambda of: ', args.spp_lambda)
    actor_critic = QPolicy(obs_shape,
        taskcla,
        base_kwargs={'F_prior': args.F_prior, 'recurrent': args.recurrent_policy}).to(device)    
else:
    from rl_module.ppo_model import Policy
    actor_critic = Policy(obs_shape,
        taskcla,
        base_kwargs={'recurrent': args.recurrent_policy}).to(device)

# Args -- Approach
if args.approach == 'fine-tuning' or args.approach == 'ft-fix':
    from rl_module.ppo import PPO as approach

    agent = approach(actor_critic,
            args.clip_param,
            args.ppo_epoch,
            args.num_mini_batch,
            args.value_loss_coef,
            args.entropy_coef,
            lr=args.lr,
            eps=args.eps,
            max_grad_norm=args.max_grad_norm,
            use_clipped_value_loss=True,
            optimizer=args.optimizer)
elif args.approach == 'ewc':
    from rl_module.ppo_ewc import PPO_EWC as approach

    agent = approach(
        actor_critic,
        args.clip_param,
        args.ppo_epoch,
        args.num_mini_batch,
        args.value_loss_coef,
        args.entropy_coef,
        lr=args.lr,
        eps=args.eps,
        max_grad_norm=args.max_grad_norm,
        use_clipped_value_loss=True,
        ewc_lambda= args.ewc_lambda,
        online = args.ewc_online,
        optimizer=args.optimizer)

elif args.approach == 'blip':
    from rl_module.ppo_blip import PPO_BLIP as approach

    agent = approach(
        actor_critic,
        args.clip_param,
        args.ppo_epoch,
        args.num_mini_batch,
        args.value_loss_coef,
        args.entropy_coef,
        lr=args.lr,
        eps=args.eps,
        max_grad_norm=args.max_grad_norm,
        use_clipped_value_loss=True,
        optimizer=args.optimizer)
    
elif args.approach == 'blip_ewc':
    from rl_module.ppo_blip_ewc import PPO_BLIP_EWC as approach

    agent = approach(
        actor_critic,
        args.clip_param,
        args.ppo_epoch,
        args.num_mini_batch,
        args.value_loss_coef,
        args.entropy_coef,
        lr=args.lr,
        eps=args.eps,
        max_grad_norm=args.max_grad_norm,
        use_clipped_value_loss=True,
        ewc_lambda= args.ewc_lambda,
        online = args.ewc_online,
        optimizer=args.optimizer)
    
elif args.approach == 'blip_spp':
    from rl_module.ppo_blip_spp import PPO_BLIP_SPP as approach

    agent = approach(
        actor_critic,
        args.clip_param,
        args.ppo_epoch,
        args.num_mini_batch,
        args.value_loss_coef,
        args.entropy_coef,
        lr=args.lr,
        eps=args.eps,
        max_grad_norm=args.max_grad_norm,
        use_clipped_value_loss=True,
        ewc_lambda= args.ewc_lambda,
        online = args.ewc_online,
        optimizer=args.optimizer, 
        loss_method=args.loss_method, 
        spp_lambda=args.spp_lambda
        )


using fisher prior of:  1e-16
using SPP lambda of:  4


Train:

In [6]:
print('Device: ', device)
print('Approach: ', args.approach)
print('Date: ', args.date)
print('Experiment: ', log_name)
print('Length task sequence: ', len(tasks_sequence))
print('Tasks: ', tasks_sequence)
print('Steps/task: ', args.num_env_steps, '\n')

tr_reward_arr = []
te_reward_arr = {}

for _type in (['mean', 'max', 'min', 'std']):
    te_reward_arr[_type] = {}
    for idx in range(len(taskcla)):
        te_reward_arr[_type]['task' + str(idx)] = []

# Define Tensorboard loggers

tb_logger_train = {
    'header':["update", "num_timesteps", "FPS", "mean_reward", "median_reward", "min_reward", "max_reward", "std", "entropy", "value_loss", "action_loss"],
    'writer':SummaryWriter(os.path.join("./result_data/", log_name, "train"))
}

tb_logger_eval = {
    'header':["update", "num_timesteps", "mean_reward", "min_reward", "max_reward", "std"],
    'writer':[]
}

for idx in range(len(taskcla)):
    tb_logger_eval['writer'].append(SummaryWriter(os.path.join("./result_data/", log_name, "eval", "task_"+str(idx))))

# Start training 

prev_total_num_steps = 0

for task_idx,env_name in tasks_sequence:
    print('Training task '+str(task_idx)+': '+env_name)
    #if task_idx == 2: args.num_env_steps = 1e6
    # renew optimizer
    agent.renew_optimizer()

    # FlatObsWrapper for MiniGrid
    envs = make_vec_envs(env_name, args.seed, args.num_processes, args.gamma, args.log_dir, device, False, wrapper_class=wrapper_class)
    obs = envs.reset()

    rollouts = RolloutStorage(args.num_steps, args.num_processes,
                                    obs_shape, envs.action_space,
                                    actor_critic.recurrent_hidden_state_size)

    rollouts.obs[0].copy_(obs)
    rollouts.to(device)

    episode_rewards = deque(maxlen=10)
    num_updates = int(args.num_env_steps) // args.num_steps // args.num_processes

    current_total_num_steps = train_ppo(actor_critic, agent, rollouts, task_idx, env_name, tasks_sequence, envs,  obs_shape, args, episode_rewards, tr_reward_arr, te_reward_arr, tb_logger_train, tb_logger_eval, num_updates, log_name, device, prev_total_num_steps, wrapper_class=wrapper_class)

    prev_total_num_steps = current_total_num_steps

    # post-processing
    if args.approach == 'fine-tuning':
        if args.single_task == True:
            envs.close()
            break
        else:
            envs.close()
    elif args.approach == 'ft-fix':
        # fix the backbone
        for param in actor_critic.features.parameters():
            param.requires_grad = False
        if args.single_task == True:
            envs.close()
            break
        else:
            envs.close()
    elif args.approach == 'ewc':
        agent.update_fisher(rollouts, task_idx)
        envs.close()
    elif args.approach == 'blip':
        agent.ng_post_processing(rollouts, task_idx)
        # save the model here so that bit allocation is saved
        save_path = os.path.join(args.save_dir, args.algo)
        torch.save(actor_critic.state_dict(),
            os.path.join(save_path, log_name + '_task_' + str(task_idx) + ".pth"))
        envs.close()
    elif args.approach == 'blip_ewc':
        agent.ng_post_processing(rollouts, task_idx)
        # save the model here so that bit allocation is saved
        save_path = os.path.join(args.save_dir, args.algo)
        torch.save(actor_critic.state_dict(),
            os.path.join(save_path, log_name + '_task_' + str(task_idx) + ".pth"))
        envs.close()
    elif args.approach == 'blip_spp':
        agent.update_omega(rollouts, task_idx)
        agent.ng_post_processing(rollouts, task_idx)
        # save the model here so that bit allocation is saved
        save_path = os.path.join(args.save_dir, args.algo)
        torch.save(actor_critic.state_dict(),
            os.path.join(save_path, log_name + '_task_' + str(task_idx) + ".pth"))
        envs.close()   

Device:  cpu
Approach:  blip_spp
Date:  2023-10-16
Experiment:  2023-10-16_minigrid-doorkey-wallgap-lavagap_blip_spp_1_500000.0_F_prior_1e-16_spp_lamb_4
Length task sequence:  3
Tasks:  [(0, 'MiniGrid-DoorKey-6x6-v0'), (1, 'MiniGrid-WallGapS6-v0'), (2, 'MiniGrid-LavaGapS6-v0')]
Steps/task:  500000.0 

Training task 0: MiniGrid-DoorKey-6x6-v0
Task 0: Evaluation using 10 episodes: mean reward 0.00000, std 0.00000 



  4%|███▋                                                                                       | 10/244 [00:51<19:23,  4.97s/it]

Updates 9, num timesteps 20480, FPS 397 
 Last 10 training episodes: mean/median reward 0.1/0.0, min/max reward 0.0/0.7, std 0.3



  8%|███████▍                                                                                   | 20/244 [01:48<21:36,  5.79s/it]

Updates 19, num timesteps 40960, FPS 376 
 Last 10 training episodes: mean/median reward 0.4/0.5, min/max reward 0.0/0.8, std 0.3



...

 94%|████████████████████████████████████████████████████████████████████████████████████▊     | 230/244 [28:52<01:30,  6.49s/it]

Updates 229, num timesteps 471040, FPS 271 
 Last 10 training episodes: mean/median reward 0.4/0.3, min/max reward 0.0/0.9, std 0.4



 98%|████████████████████████████████████████████████████████████████████████████████████████▌ | 240/244 [30:01<00:29,  7.33s/it]

Updates 239, num timesteps 491520, FPS 272 
 Last 10 training episodes: mean/median reward 0.1/0.0, min/max reward 0.0/0.9, std 0.3



100%|██████████████████████████████████████████████████████████████████████████████████████████| 244/244 [30:29<00:00,  7.50s/it]


Save full model (with last task training)

In [12]:
torch.save(actor_critic,
            os.path.join(save_path, log_name + '_fullmodel_task_' + str(task_idx) + ".pth"))

Test load full model

In [ ]:
actor_critic_test = torch.load(os.path.join(save_path, log_name + '_fullmodel_task_' + str(task_idx) + ".pth"))

Evaluate

In [7]:
from rl_module.evaluation import evaluate
args.seed = 1
print(tasks_sequence)
task_idx = tasks_sequence[-1][0]
ob_rms = None

print('Evaluating tasks:')
tot_eval_episode_mean_rewards = []

eval_episode_mean_rewards = evaluate(actor_critic, ob_rms, tasks_sequence, args.seed,
                            args.num_processes, args.log_dir, device, obs_shape, task_idx, args.gamma, wrapper_class=wrapper_class, episodes=100)
tot_eval_episode_mean_rewards.append(eval_episode_mean_rewards)

[(0, 'MiniGrid-DoorKey-6x6-v0'), (1, 'MiniGrid-WallGapS6-v0'), (2, 'MiniGrid-LavaGapS6-v0')]
Evaluating tasks:
Task 0: Evaluation using 100 episodes: mean reward 0.96388, std 0.01217 

Task 1: Evaluation using 100 episodes: mean reward 0.93356, std 0.03848 

Task 2: Evaluation using 100 episodes: mean reward 0.00000, std 0.00000 



In [8]:
from rl_module.evaluation import evaluate

print(tasks_sequence)
task_idx = tasks_sequence[-1][0]
ob_rms = None
seed_list = [100,200,300]

print('Evaluating tasks:')
tot_eval_episode_mean_rewards = []
for i in range(3):
    eval_episode_mean_rewards = evaluate(actor_critic, ob_rms, tasks_sequence, seed_list[i],
                                args.num_processes, args.log_dir, device, obs_shape, task_idx, args.gamma, wrapper_class=wrapper_class, episodes=100)
    tot_eval_episode_mean_rewards.append(eval_episode_mean_rewards)

# Calculating the median of the means by column
median_of_means = np.median(np.array([d['mean'] for d in tot_eval_episode_mean_rewards]), axis=0)

# print('Final evaluation')
for i, value in enumerate(median_of_means):
    print('Task ',i,': median reward (3 eval) ', value)                               

[(0, 'MiniGrid-DoorKey-6x6-v0'), (1, 'MiniGrid-WallGapS6-v0'), (2, 'MiniGrid-LavaGapS6-v0')]
Evaluating tasks:
Task 0: Evaluation using 100 episodes: mean reward 0.57173, std 0.45626 

Task 1: Evaluation using 100 episodes: mean reward 0.93806, std 0.03182 

Task 2: Evaluation using 100 episodes: mean reward 0.00000, std 0.00000 

Task 0: Evaluation using 100 episodes: mean reward 0.39217, std 0.45813 

Task 1: Evaluation using 100 episodes: mean reward 0.94313, std 0.01960 

Task 2: Evaluation using 100 episodes: mean reward 0.00000, std 0.00000 

Task 0: Evaluation using 100 episodes: mean reward 0.96495, std 0.01054 

Task 1: Evaluation using 100 episodes: mean reward 0.79344, std 0.31205 

Task 2: Evaluation using 100 episodes: mean reward 0.00000, std 0.00000 

Task  0 : median reward (3 eval)  0.571725
Task  1 : median reward (3 eval)  0.9380625
Task  2 : median reward (3 eval)  0.0


Check MiniGrid vectorized environments

In [ ]:
import gym
from gym import spaces
from gym.wrappers import Monitor
import gym_minigrid
from gym_minigrid.wrappers import FlatObsWrapper, ImgObsWrapper, RGBImgPartialObsWrapper, RGBImgObsWrapper
from gym.wrappers import Monitor

In [ ]:
import matplotlib.pyplot as plt

env_name = 'MiniGrid-DoorKey-6x6-v0'

# Create vectorized environment with wrapper class
vec_env = make_vec_envs(env_name, args.seed, args.num_processes, args.gamma, args.log_dir, device, False, wrapper_class=FlatObsWrapper)

# Plot snapshot of vectorized environment and check randomized init
vec_env.reset()
before_img = vec_env.render('rgb_array')

plt.figure(figsize = (6.,6.))
plt.imshow(before_img);